<a href="https://colab.research.google.com/github/stanleykywu/Fake_News_Processing/blob/master/Fake_News_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install Tensorflow
#!pip install Keras
#!pip install nltk

In [0]:
#from google.colab import drive
#drive.mount("/content/drive")

In [0]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

def clean_string(str):
  arr = np.array([word for word in str.split() if word not in stoplist])
  y=""
  for i in range(arr.size):
    y += arr[i] + " "
  return y

In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Fake_News_Language_Processing/Fake.csv", usecols=['title'])
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7

train_fake = np.array(df[msk])[:,0]
train_fake_output = np.array([1 for i in range(train_fake.size)])
test_fake = np.array(df[~msk])[:,0]
test_fake_output = np.array([1 for i in range(test_fake.size)])

In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Fake_News_Language_Processing/True.csv", usecols=['title'])
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7
train_true = np.array(df[msk])[:,0]
train_true_output = np.array([0 for i in range(train_true.size)])
test_true = np.array(df[~msk])[:,0]
test_true_output = np.array([0 for i in range(test_true.size)])

In [0]:
train_string = np.append(train_fake, train_true)
train_output = np.append(train_fake_output, train_true_output)

test_string = np.append(test_fake, test_true)
test_output = np.append(test_fake_output, test_true_output)

for i in range(train_string.size):
  train_string[i] = clean_string(train_string[i])

In [0]:
# Pre-processing String inputs by transforming them into integers 
# by using hashing_trick
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import hashing_trick

length = 750

train_data = np.array([np.array(hashing_trick(train_string[i], length)) for i in range(train_string.size)])
test_data = np.array([np.array(hashing_trick(test_string[i], length)) for i in range(test_string.size)])

In [371]:
flattened = np.hstack(train_data)
most_freq = []
for i in range(20):
  counts = np.bincount(flattened)
  max = np.argmax(counts)
  # print(max)
  most_freq.append(max)
  x = np.array([flattened[i] != max for i in range(flattened.size)])
  flattened = flattened[x]

print(most_freq)

[43, 187, 190, 612, 53, 124, 45, 222, 92, 489, 706, 589, 537, 614, 193, 154, 638, 480, 213, 260]


In [372]:
for i in range(train_data.size):
  x = np.array([train_data[i][j] in most_freq for j in range(train_data[i].size)])
  train_data[i] = train_data[i][x]

resized_training = np.array([np.resize(np.asarray(train_data[i]), (5)) for i in range(train_data.size)])

[ 43 193 213  43 193]
[43 43 43 43 43]


In [373]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(5, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(5, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(5, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(resized_training, train_output.T, epochs=5)

Epoch 1/5
983/983 [==============================] - 1s 1ms/step - loss: 0.7192 - accuracy: 0.6246
Epoch 2/5
983/983 [==============================] - 1s 1ms/step - loss: 0.6074 - accuracy: 0.6719
Epoch 3/5
983/983 [==============================] - 1s 1ms/step - loss: 0.5882 - accuracy: 0.6919
Epoch 4/5
983/983 [==============================] - 1s 1ms/step - loss: 0.5788 - accuracy: 0.6976
Epoch 5/5
983/983 [==============================] - 1s 2ms/step - loss: 0.5704 - accuracy: 0.7024
